In [16]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [17]:
df = pd.read_csv("src/data/analyzed/monthly_demand_clean.csv")


In [18]:
#sum total; industry etc to get a European value
df_europe = df.groupby(['type', 'year', 'month' ])['demand'].sum().reset_index()
df_europe['country'] = 'Europe'  # Assign country as "Europe"



In [ ]:
# Define a mapping of EU country codes to their full names
country_mapping = {
    "AT": "Austria", "BE": "Belgium", "BG": "Bulgaria", "HR": "Croatia", 
    "CY": "Cyprus", "CZ": "Czech Republic", "DK": "Denmark", "EE": "Estonia", 
    "FI": "Finland", "FR": "France", "DE": "Germany", "GR": "Greece", 
    "HU": "Hungary", "IS": "Iceland", "IE": "Ireland", "IT": "Italy", 
    "LV": "Latvia", "LI": "Liechtenstein", "LT": "Lithuania", "LU": "Luxembourg", 
    "MT": "Malta", "NL": "Netherlands", "NO": "Norway", "PL": "Poland", 
    "PT": "Portugal", "RO": "Romania", "SK": "Slovakia", "SI": "Slovenia", 
    "ES": "Spain", "SE": "Sweden", "CH": "Switzerland", "TR": "Turkey", 
    "UK": "United Kingdom", "Europe": "Europe*"
}

# Replace country codes with full names
df["country_full"] = df["country"].map(country_mapping)
df["country"] = df["country_full"]

# Create the 'countryType' column with full country names
df["country_type"] = df["country_full"] + " - " + df["type"]

In [19]:
df = pd.concat([df, df_europe], ignore_index=True)

In [20]:
# Define the years to average
years_to_average = [2019, 2020, 2021]

# Filter for rows corresponding to the specified years
filtered_df = df[df['year'].isin(years_to_average)]

# Group by country, type, and month, then calculate the average demand
index_df = (
    filtered_df
    .groupby(['country', 'type', 'month'])['demand']
    .mean()
    .reset_index(name='monthly_index')  # Calculate the average demand
)


df= df.merge(index_df_s, on=['country', 'type', 'month'], how='left')




# Indexed

In [ ]:

df_i = df.copy()
index_df_i = index_df.copy()

In [ ]:
df_i= df.rename(columns={'demand_x': 'demand'})
df_i = df.drop(columns=['demand_y'], errors='ignore')


# Update the year column:
# - Use year_y if year_x is in [2019, 2020, 2021]
# - Otherwise, use year_x
df_i['year'] = df_i['year_y'].where(df_i['year_x'].isin(years_to_average), df['year_x'])


# Remove rows with years 2019, 2020, or 2021
df_i = df_i[~df_i['year_x'].isin(years_to_average)]

df_i = df_i.drop(columns=['year_x', 'year_y'])


# Avoid dividing by itself for the "AVG-2019-2021" rows (set to 1 for these rows)
#df.loc[df['year'] == "AVG-2019-2021", 'indexed_demand'] = 1

df_i = pd.concat([df_i, index_df_i], ignore_index=True)
# Calculate indexed demand values for all rows except "AVG-2019-2021"
df_i['demand_indexed'] = (df_i['demand'] / df_i['monthly_index'])*100

# Drop rows where the 'year' column equals "AVG-2019-2021"
df_i = df_i[df_i['year'] != "AVG-2019-2021"]

# Optionally, print the updated DataFrame or save it
print("Rows with 'year' == 'AVG-2019-2021' have been removed.")
df_i['date'] = pd.to_datetime(df_i[['year', 'month']].assign(day=15))
# Ensure 'month' and 'year' columns are strings before concatenation
df_i["month"] = df_i["month"].astype(str).str.zfill(2)  # Add leading zero to month if needed
df_i["year"] = df_i["year"].astype(str)

# Create the new column 'monthb' as a concatenation of 'month' and 'year'
df_i["monthb"] = df_i["month"] + "/" + df_i["year"]

#Drop type=total
df_i = df_i[df_i['type'] != "total"]

# Indexed by sector (stacked bar)

In [ ]:
df_s = ds.copy()
index_df_s = index_df.copy()

In [ ]:
# Calculate indexed demand values for all rows except "AVG-2019-2021"
df_s['demand_sector'] = (df_s['demand'] - df_s['monthly_index'])

In [22]:

# Remove rows with years 2019, 2020, or 2021
df_s= df_s[~df_s['year'].isin(years_to_average)]

# Assuming index_df and df are predefined DataFrames
# Add a new column for the year and set it to "2021"
index_df_s['year'] = "2021"  # Changed to "AVG-2019-2021" based on your requirement
index_df_s['month'] = 12

# Set indexed_demand to 1 for the new rows
index_df_s['monthly_index'] = 1


# Set indexed_demand explicitly to 100 for all rows in index_df
index_df_s['demand_indexed'] = 100

# Keep unique rows based on 'country' and 'type'
unique_index_df = index_df_s.drop_duplicates(subset=['country', 'type'])

# Concatenate unique rows with the original DataFrame
df_s = pd.concat([unique_index_df, df], ignore_index=True)
df_s["month"] = df_s["month"].astype(str).str.zfill(2)  # Add leading zero to month if needed
df_s["year"] = df_s["year"].astype(str)

# Create the new column 'monthb' as a concatenation of 'month' and 'year'
df_s["x_value"] =  df_s["month"] + "/"+ df_s["year"]


# Save .json

In [1]:
df_json = df_s.copy()
df_json["y_value"] = df_s.pop("demand_sector")
df_json["group_value"] = df_s.pop("type")
df_json["group_b_value"] = df_s.pop("country")
# Save to a JSON file
file_path = "highcharts/data/monthly_demand_sector.json"
df_json.to_json(file_path, orient='records', indent=4)
print(f"The file has been saved as: {file_path}")


NameError: name 'df_s' is not defined

In [25]:
df_json = df
df_json["y_value"] = df.pop("demand_indexed")
df_json["group_value"] = df.pop("country_type")
# Save to a JSON file
file_path = "highcharts/data/monthly_demand_indexed.json"
df_json.to_json(file_path, orient='records', indent=4)
print(f"The file has been saved as: {file_path}")

The file has been saved as: src/data/analyzed/monthly_demand_clean.json
